In [1]:
import os
import pandas as pd
import regex as re
import spacy
from collections import defaultdict, Counter
import numpy as np

In [2]:
from spacy.lang.pl import Polish
nlp = Polish()
tokenizer = nlp.tokenizer

In [3]:
directory = 'ustawy'
def load_bills(directory):
    bills = {}
    for file_name in os.listdir(directory):
        path = os.path.join(directory, file_name)
        if os.path.isfile(path):
            with open(path, encoding='utf-8') as f:
                content = f.read()
                content = content.replace("\xa0", " ").replace("\xad", "")
                content = re.sub(r"\s+", " ", content)
                content = content.lower()
                bills[file_name] = content
    return bills
bills = load_bills(directory)

### Use SpaCy tokenizer API to tokenize the text from the law corpus.

In [4]:
tokenized_bills = {name: [token.text for token in tokenizer(content)] for name, content in bills.items()}

In [5]:
tokenized_bills['2004_1264.txt']

[' ',
 'dz.u',
 '.',
 'z',
 '2004',
 'r',
 '.',
 'nr',
 '121',
 ',',
 'poz',
 '.',
 '1264',
 'u',
 's',
 't',
 'a',
 'w',
 'a',
 'z',
 'dnia',
 '20',
 'kwietnia',
 '2004',
 'r',
 '.',
 'o',
 'zmianie',
 'ustawy',
 'o',
 'emeryturach',
 'i',
 'rentach',
 'z',
 'funduszu',
 'ubezpieczeń',
 'społecznych',
 'oraz',
 'niektórych',
 'innych',
 'ustaw[1',
 ']',
 'art',
 '.',
 '1',
 '.',
 'w',
 'ustawie',
 'z',
 'dnia',
 '17',
 'grudnia',
 '1998',
 'r',
 '.',
 'o',
 'emeryturach',
 'i',
 'rentach',
 'z',
 'funduszu',
 'ubezpieczeń',
 'społecznych',
 '(',
 'dz.u',
 '.',
 'z',
 '2004',
 'r',
 '.',
 'nr',
 '39',
 ',',
 'poz',
 '.',
 '353',
 ')',
 'wprowadza',
 'się',
 'następujące',
 'zmiany',
 ':',
 '1',
 ')',
 'w',
 'art',
 '.',
 '4',
 ':',
 'a',
 ')',
 'po',
 'pkt',
 '4',
 'dodaje',
 'się',
 'pkt',
 '4a',
 'w',
 'brzmieniu',
 ':',
 '"',
 '4a',
 ')',
 'kwota',
 'rocznej',
 'podstawy',
 'wymiaru',
 'składek',
 'na',
 'ubezpieczenia',
 'emerytalne',
 'i',
 'rentowe',
 '-',
 'roczną',
 'podstawę',

### Compute bigram counts of downcased tokens.

In [6]:
def create_ngram(text, n = 2):  
    result = []
    count = 0
    for token in text[:len(text)-n+1]:  
        result.append(tuple(text[count:count+n]))
        count += 1  
    return result

In [7]:
bigrams_global = []

for _, content in tokenized_bills.items():   
    bigrams_global.extend(create_ngram(content))

In [8]:
bigrams_global_count = Counter(bigrams_global)

In [9]:
bigrams_global_count.most_common(10)

[(('art', '.'), 83778),
 (('ust', '.'), 53552),
 (('poz', '.'), 45198),
 ((',', 'poz'), 43188),
 (('.', '1'), 39927),
 (('-', '-'), 36547),
 (('r', '.'), 33010),
 (('w', 'art'), 32043),
 ((',', 'o'), 29926),
 (('mowa', 'w'), 28471)]

### Discard bigrams containing characters other than letters. Make sure that you discard the invalid entries after computing the bigram counts.

In [10]:
bigrams_global_clean = defaultdict(int)

for key, value in bigrams_global_count.items():
    if key[0].isalpha() and key[1].isalpha():
        bigrams_global_clean[key] = value

In [11]:
bigrams_global_clean

defaultdict(int,
            {('ustawa', 'z'): 1190,
             ('z', 'dnia'): 9527,
             ('o', 'zmianie'): 1299,
             ('zmianie', 'ustawy'): 850,
             ('ustawy', 'o'): 1444,
             ('o', 'podatku'): 533,
             ('podatku', 'od'): 382,
             ('od', 'towarów'): 273,
             ('towarów', 'i'): 530,
             ('i', 'usług'): 565,
             ('usług', 'oraz'): 122,
             ('oraz', 'o'): 945,
             ('podatku', 'akcyzowym'): 75,
             ('akcyzowym', 'art'): 15,
             ('w', 'ustawie'): 5180,
             ('ustawie', 'z'): 3661,
             ('i', 'nr'): 8435,
             ('wprowadza', 'się'): 1849,
             ('się', 'następujące'): 1855,
             ('następujące', 'zmiany'): 1808,
             ('w', 'art'): 32043,
             ('dotychczasowa', 'treść'): 37,
             ('treść', 'otrzymuje'): 37,
             ('otrzymuje', 'oznaczenie'): 41,
             ('oznaczenie', 'ust'): 62,
             ('dodaje', '

### Use pointwise mutual information to compute the measure for all pairs of words.

In [12]:
words_global = []

for _, content in tokenized_bills.items():
    for word in content:
        if word.isalpha():
            words_global.append(word)

In [13]:
words_global_counter = Counter(words_global)

In [14]:
number_of_words = sum(words_global_counter.values())
number_of_bigrams = sum(bigrams_global_clean.values())
(number_of_words, number_of_bigrams)

(3568022, 2756037)

In [15]:
pmi = defaultdict(float)

for bigram, value in bigrams_global_clean.items():
    pmi[bigram] = np.log2( (value/number_of_bigrams) / ((words_global_counter[bigram[0]]/number_of_words) * (words_global_counter[bigram[1]]/number_of_words) ))

### Sort the word pairs according to that measure in the descending order and determine top 10 entries.

In [16]:
pmi_sorted = sorted(pmi.items(), key=lambda x:x[1], reverse = True)

In [17]:
pd.DataFrame.from_dict(pmi_sorted)

,0,1
0,"(kołowe, jednoosiowe)",22.139222
1,"(zbrojeń, żelbeto)",22.139222
2,"(prefabrykatów, wnętrzowe)",22.139222
3,"(gołe, aluminiowe)",22.139222
4,"(polistyrenu, spienionego)",22.139222
...,...,...
524688,"(w, o)",-10.462220
524689,"(w, oraz)",-10.513418
524690,"(w, lub)",-10.962108
524691,"(na, w)",-11.107124


In [18]:
pmi_sorted[:10]

[(('kołowe', 'jednoosiowe'), 22.139222334408554),
 (('zbrojeń', 'żelbeto'), 22.139222334408554),
 (('prefabrykatów', 'wnętrzowe'), 22.139222334408554),
 (('gołe', 'aluminiowe'), 22.139222334408554),
 (('polistyrenu', 'spienionego'), 22.139222334408554),
 (('objaśnieniem', 'figur'), 22.139222334408554),
 (('wkładzie', 'wnoszonym'), 22.139222334408554),
 (('doktorem', 'habilitowanym'), 22.139222334408554),
 (('losy', 'loteryjne'), 22.139222334408554),
 (('uw', 'zględnieniu'), 22.139222334408554)]

### Filter bigrams with number of occurrences lower than 5. Determine top 10 entries for the remaining dataset (>=5 occurrences).

In [19]:
bigrams_filtered = defaultdict(int)

for k, v in bigrams_global_clean.items():
    if v >= 5:
        bigrams_filtered[k] = v

In [20]:
bigrams_filtered

defaultdict(int,
            {('ustawa', 'z'): 1190,
             ('z', 'dnia'): 9527,
             ('o', 'zmianie'): 1299,
             ('zmianie', 'ustawy'): 850,
             ('ustawy', 'o'): 1444,
             ('o', 'podatku'): 533,
             ('podatku', 'od'): 382,
             ('od', 'towarów'): 273,
             ('towarów', 'i'): 530,
             ('i', 'usług'): 565,
             ('usług', 'oraz'): 122,
             ('oraz', 'o'): 945,
             ('podatku', 'akcyzowym'): 75,
             ('akcyzowym', 'art'): 15,
             ('w', 'ustawie'): 5180,
             ('ustawie', 'z'): 3661,
             ('i', 'nr'): 8435,
             ('wprowadza', 'się'): 1849,
             ('się', 'następujące'): 1855,
             ('następujące', 'zmiany'): 1808,
             ('w', 'art'): 32043,
             ('dotychczasowa', 'treść'): 37,
             ('treść', 'otrzymuje'): 37,
             ('otrzymuje', 'oznaczenie'): 41,
             ('oznaczenie', 'ust'): 62,
             ('dodaje', '

In [21]:
pmi_filtered = defaultdict(float)

for bigram, value in bigrams_filtered.items():
    pmi_filtered[bigram] = np.log2( (value/number_of_bigrams) / ((words_global_counter[bigram[0]]/number_of_words) * (words_global_counter[bigram[1]]/number_of_words) ))
pmi_filtered_sorted = sorted(pmi_filtered.items(), key=lambda x:x[1], reverse = True)

In [22]:
pmi_filtered_sorted[:10]

[(('świeckie', 'przygotowujące'), 19.817294239521193),
 (('klęskami', 'żywiołowymi'), 19.817294239521193),
 (('ręcznego', 'miotacza'), 19.817294239521193),
 (('stajnią', 'wyścigową'), 19.817294239521193),
 (('otworami', 'wiertniczymi'), 19.817294239521193),
 (('teryto', 'rialnego'), 19.817294239521193),
 (('obcowania', 'płciowego'), 19.817294239521193),
 (('nietykalność', 'cielesną'), 19.817294239521193),
 (('młyny', 'kulowe'), 19.817294239521193),
 (('młynki', 'młotkowe'), 19.817294239521193)]

### Use KRNNT or Clarin-PL API(https://ws.clarin-pl.eu/tager.shtml) to tag and lemmatize the corpus.

In [23]:
import xmltodict

CLARIN_PATH = 'clarin'
def load_clarin_bills(directory):
    clarin_tokens = {}
    for file_name in os.listdir(directory):
        print(file_name)
        name = re.search(r'(\d+.*).txt', file_name).group(1)
        path = os.path.join(directory, file_name)
        if os.path.isfile(path):
            tokens_per_file = []
            with open(path, mode='r', encoding = 'utf-8') as f:
                bill_clarin = xmltodict.parse(f.read())
                for sentence in bill_clarin['chunkList']['chunk']:
                    for token in sentence['sentence']['tok']:
                        try:
                            if isinstance(token['lex'], list):
                                tokens_per_file.append(f"{token['lex'][0]['base'].lower()}:{token['lex'][0]['ctag'].split(':')[0].lower()}")
                            else:
                                tokens_per_file.append(f"{token['lex']['base'].lower()}:{token['lex']['ctag'].split(':')[0].lower()}")
                        except TypeError:
                            pass
                clarin_tokens[name] = tokens_per_file
    return clarin_tokens

In [24]:
clarin_data = load_clarin_bills(CLARIN_PATH)

ustawy%1993_599.txt
ustawy%1993_602.txt
ustawy%1993_645.txt
ustawy%1993_646.txt
ustawy%1994_150.txt
ustawy%1994_195.txt
ustawy%1994_201.txt
ustawy%1994_214.txt
ustawy%1994_215.txt
ustawy%1994_288.txt
ustawy%1994_294.txt
ustawy%1994_331.txt
ustawy%1994_332.txt
ustawy%1994_344.txt
ustawy%1994_362.txt
ustawy%1994_363.txt
ustawy%1994_368.txt
ustawy%1994_369.txt
ustawy%1994_37.txt
ustawy%1994_384.txt
ustawy%1994_385.txt
ustawy%1994_395.txt
ustawy%1994_396.txt
ustawy%1994_419.txt
ustawy%1994_472.txt
ustawy%1994_473.txt
ustawy%1994_516.txt
ustawy%1994_536.txt
ustawy%1994_591.txt
ustawy%1994_592.txt
ustawy%1994_600.txt
ustawy%1994_601.txt
ustawy%1994_602.txt
ustawy%1994_615.txt
ustawy%1994_627.txt
ustawy%1994_645.txt
ustawy%1994_669.txt
ustawy%1994_670.txt
ustawy%1994_704.txt
ustawy%1994_97.txt
ustawy%1995_1.txt
ustawy%1995_121.txt
ustawy%1995_141.txt
ustawy%1995_143.txt
ustawy%1995_152.txt
ustawy%1995_164.txt
ustawy%1995_184.txt
ustawy%1995_2.txt
ustawy%1995_208.txt
ustawy%1995_221.txt
ustawy

ustawy%1998_759.txt
ustawy%1998_931.txt
ustawy%1998_937.txt
ustawy%1998_938.txt
ustawy%1999_1000.txt
ustawy%1999_1001.txt
ustawy%1999_1063.txt
ustawy%1999_1065.txt
ustawy%1999_1100.txt
ustawy%1999_1101.txt
ustawy%1999_1107.txt
ustawy%1999_1151.txt
ustawy%1999_1177.txt
ustawy%1999_1227.txt
ustawy%1999_1231.txt
ustawy%1999_1232.txt
ustawy%1999_1233.txt
ustawy%1999_1234.txt
ustawy%1999_1235.txt
ustawy%1999_1236.txt
ustawy%1999_1255.txt
ustawy%1999_1256.txt
ustawy%1999_1279.txt
ustawy%1999_1280.txt
ustawy%1999_1281.txt
ustawy%1999_135.txt
ustawy%1999_136.txt
ustawy%1999_154.txt
ustawy%1999_170.txt
ustawy%1999_303.txt
ustawy%1999_304.txt
ustawy%1999_305.txt
ustawy%1999_309.txt
ustawy%1999_310.txt
ustawy%1999_32.txt
ustawy%1999_400.txt
ustawy%1999_439.txt
ustawy%1999_440.txt
ustawy%1999_461.txt
ustawy%1999_483.txt
ustawy%1999_484.txt
ustawy%1999_485.txt
ustawy%1999_486.txt
ustawy%1999_527.txt
ustawy%1999_528.txt
ustawy%1999_530.txt
ustawy%1999_532.txt
ustawy%1999_548.txt
ustawy%1999_596.txt


ustawy%2002_1182.txt
ustawy%2002_1204.txt
ustawy%2002_1267.txt
ustawy%2002_1523.txt
ustawy%2002_1526.txt
ustawy%2002_1569.txt
ustawy%2002_1662.txt
ustawy%2002_1681.txt
ustawy%2002_1685.txt
ustawy%2002_1689.txt
ustawy%2002_1731.txt
ustawy%2002_1763.txt
ustawy%2002_1800.txt
ustawy%2002_1801.txt
ustawy%2002_1802.txt
ustawy%2002_1803.txt
ustawy%2002_1920.txt
ustawy%2002_1923.txt
ustawy%2002_1937.txt
ustawy%2002_1954.txt
ustawy%2002_731.txt
ustawy%2002_90.txt
ustawy%2002_945.txt
ustawy%2002_976.txt
ustawy%2003_1029.txt
ustawy%2003_1031.txt
ustawy%2003_1039.txt
ustawy%2003_1061.txt
ustawy%2003_1065.txt
ustawy%2003_1067.txt
ustawy%2003_1068.txt
ustawy%2003_1069.txt
ustawy%2003_1122.txt
ustawy%2003_1123.txt
ustawy%2003_1176.txt
ustawy%2003_1186.txt
ustawy%2003_1189.txt
ustawy%2003_1268.txt
ustawy%2003_1300.txt
ustawy%2003_1302.txt
ustawy%2003_1303.txt
ustawy%2003_1304.txt
ustawy%2003_1324.txt
ustawy%2003_1325.txt
ustawy%2003_1450.txt
ustawy%2003_1452.txt
ustawy%2003_1453.txt
ustawy%2003_1454.t

### Using the tagged corpus compute bigram statistic for the tokens containing: a. lemmatized, downcased word b. morphosyntactic category of the word (subst, fin, adj, etc.)

In [25]:
clarin_data['1993_599']

['dzieje_(apostolskie):brev',
 '.:interp',
 'u:prep',
 '.:interp',
 'z:prep',
 '1993:num',
 'r:ign',
 '.:interp',
 'nr:subst',
 '129:num',
 ',:interp',
 'poz:ign',
 '.:interp',
 '599:num',
 'ustawa:subst',
 'z:prep',
 'dzień:subst',
 '9:num',
 'grudzień:subst',
 '1993:num',
 'r:ign',
 '.:interp',
 'o:prep',
 'zmiana:subst',
 'ustawa:subst',
 'o:prep',
 'podatek:subst',
 'od:prep',
 'towar:subst',
 'i:conj',
 'usługa:subst',
 'oraz:conj',
 'o:prep',
 'podatek:subst',
 'akcyzowy:adj',
 'art:ign',
 '.:interp',
 '1:num',
 '.:interp',
 'w:prep',
 'ustawa:subst',
 'z:prep',
 'dzień:subst',
 '8:num',
 'styczeń:subst',
 '1993:num',
 'r:ign',
 '.:interp',
 'o:prep',
 'podatek:subst',
 'od:prep',
 'towar:subst',
 'i:conj',
 'usługa:subst',
 'oraz:conj',
 'o:prep',
 'podatek:subst',
 'akcyzowy:adj',
 '(:interp',
 'dzieje_(apostolskie):brev',
 '.:interp',
 'u:prep',
 '.:interp',
 'nr:subst',
 '11:num',
 ',:interp',
 'poz:ign',
 '.:interp',
 '50:num',
 'i:conj',
 'nr:subst',
 '28:num',
 ',:interp',

In [26]:
bigrams_clarin = []
for _, content in clarin_data.items():
    bigrams_clarin.extend(create_ngram(content))

In [27]:
bigrams_clarin_clean = []
for item in bigrams_clarin:
    if item[0].split(':')[0].isalpha() and item[1].split(':')[0].isalpha():
        bigrams_clarin_clean.append(item)

In [28]:
bigrams_clarin_clean_counter = Counter(bigrams_clarin_clean)
bigrams_clarin_clean_counter.most_common(10)

[(('w:prep', 'art:ign'), 32044),
 (('o:prep', 'który:adj'), 28656),
 (('który:adj', 'mowa:subst'), 28538),
 (('mowa:subst', 'w:prep'), 28473),
 (('w:prep', 'usta:subst'), 23557),
 (('z:prep', 'dzień:subst'), 11360),
 (('otrzymywać:fin', 'brzmienie:subst'), 10536),
 (('określony:adj', 'w:prep'), 10240),
 (('do:prep', 'sprawa:subst'), 8718),
 (('ustawa:subst', 'z:prep'), 8624)]

In [29]:
tokens_clarin_global = []

for _, content in clarin_data.items():
    for word in content:
        if word.split(':')[0].isalpha():
            tokens_clarin_global.append(word)

In [30]:
tokens_clarin_global_counter = Counter(tokens_clarin_global)
tokens_clarin_global_counter.most_common(10)

[('w:prep', 202949),
 ('i:conj', 90044),
 ('z:prep', 87991),
 ('art:ign', 83805),
 ('o:prep', 64807),
 ('do:prep', 60768),
 ('usta:subst', 53641),
 ('na:prep', 50657),
 ('który:adj', 49382),
 ('się:qub', 45887)]

### Compute the same statistics as for the non-lemmatized words (i.e. PMI) and print top-10 entries with at least 5 occurrences.

In [31]:
number_of_tokens_clarin = sum(tokens_clarin_global_counter.values())
number_of_bigrams_clarin = sum(bigrams_clarin_clean_counter.values())

In [32]:
bigrams_filtered_clarin = defaultdict(int)

for k, v in bigrams_clarin_clean_counter.items():
    if v >= 5:
        bigrams_filtered_clarin[k] = v

In [33]:
pmi_filtered_clarin = defaultdict(float)

for bigram, value in bigrams_filtered_clarin.items():
    pmi_filtered_clarin[bigram] = np.log2( (value/number_of_bigrams_clarin) / ((tokens_clarin_global_counter[bigram[0]]/number_of_tokens_clarin) * (tokens_clarin_global_counter[bigram[1]]/number_of_tokens_clarin) ))
pmi_filtered_clarin_sorted = sorted(pmi_filtered_clarin.items(), key=lambda x:x[1], reverse = True)

In [34]:
pmi_filtered_clarin_sorted[:10]

[(('teryto:ign', 'rialnego:ign'), 19.823458847837117),
 (('młynek:subst', 'młotkowy:adj'), 19.823458847837117),
 (('grzegorz:subst', 'schetyna:ign'), 19.823458847837117),
 (('pasta:subst', 'emulsyjny:adj'), 19.560424442003324),
 (('odpowiedzieć:fin', 'dzialności:ign'), 19.560424442003324),
 (('łańcuchowa:subst', 'rozszczepienie:subst'), 19.560424442003324),
 (('chrom:subst', 'sześciowartościowy:adj'), 19.560424442003324),
 (('adam:subst', 'mickiewicz:subst'), 19.560424442003324),
 (('piotrek:subst', 'trybunalski:adj'), 19.338032020666876),
 (('młyn:subst', 'kulowy:adj'), 19.338032020666876)]

### Compute trigram counts for both corpora and perform the same filtering.

#### Spacy

In [35]:
trigrams_global = []

for _, content in tokenized_bills.items():   
    trigrams_global.extend(create_ngram(content, 3))

In [36]:
trigrams_global_count = Counter(trigrams_global)
trigrams_global_count.most_common(10)

[((',', 'poz', '.'), 43166),
 (('-', '-', '-'), 34646),
 (('w', 'art', '.'), 32033),
 (('w', 'ust', '.'), 23520),
 (('ust', '.', '1'), 23324),
 (('.', 'art', '.'), 22917),
 (('r', '.', 'nr'), 17856),
 (('_', '_', '_'), 16213),
 (('.', '1', '.'), 15609),
 (('.', '2', '.'), 15279)]

In [37]:
trigrams_global_clean = defaultdict(int)

for key, value in trigrams_global_count.items():
    if key[0].isalpha() and key[1].isalpha() and key[2].isalpha():
        trigrams_global_clean[key] = value

In [38]:
number_of_trigrams = sum(trigrams_global_clean.values())
number_of_words_trigrams = number_of_words

In [39]:
trigrams_filtered = defaultdict(int)

for k, v in trigrams_global_clean.items():
    if v >= 5:
        trigrams_filtered[k] = v

In [40]:
trigrams_filtered

defaultdict(int,
            {('ustawa', 'z', 'dnia'): 1190,
             ('o', 'zmianie', 'ustawy'): 849,
             ('zmianie', 'ustawy', 'o'): 642,
             ('ustawy', 'o', 'podatku'): 135,
             ('o', 'podatku', 'od'): 124,
             ('podatku', 'od', 'towarów'): 226,
             ('od', 'towarów', 'i'): 257,
             ('towarów', 'i', 'usług'): 445,
             ('i', 'usług', 'oraz'): 98,
             ('usług', 'oraz', 'o'): 57,
             ('oraz', 'o', 'podatku'): 53,
             ('o', 'podatku', 'akcyzowym'): 71,
             ('podatku', 'akcyzowym', 'art'): 15,
             ('w', 'ustawie', 'z'): 3645,
             ('ustawie', 'z', 'dnia'): 3649,
             ('wprowadza', 'się', 'następujące'): 1804,
             ('się', 'następujące', 'zmiany'): 1806,
             ('dotychczasowa', 'treść', 'otrzymuje'): 37,
             ('treść', 'otrzymuje', 'oznaczenie'): 37,
             ('otrzymuje', 'oznaczenie', 'ust'): 16,
             ('dodaje', 'się', 'ust'): 

In [49]:
pmi_trigrams = defaultdict(float)

for trigram, value in trigrams_filtered.items():
    denominator = 1
    for token in trigram:
        denominator *=  words_global_counter[token] / number_of_words_trigrams
    denominator *= 2 ** pmi_filtered[(trigram[0], trigram[1])]

    pmi_trigrams[trigram] = pmi_filtered[(trigram[0], trigram[1])] + np.log2((value/number_of_trigrams) / denominator)

pmi_trigrams_sorted = sorted(pmi_trigrams.items(), key=lambda x:x[1], reverse = True)

#### Clarin

In [50]:
trigrams_clarin = []
for _, content in clarin_data.items():
    trigrams_clarin.extend(create_ngram(content, 3))

In [51]:
trigrams_clarin_clean = []
for item in trigrams_clarin:
    if item[0].split(':')[0].isalpha() and item[1].split(':')[0].isalpha() and item[2].split(':')[0].isalpha():
        trigrams_clarin_clean.append(item)

In [52]:
trigrams_clarin_clean_counter = Counter(trigrams_clarin_clean)
trigrams_clarin_clean_counter.most_common(10)

[(('o:prep', 'który:adj', 'mowa:subst'), 28535),
 (('który:adj', 'mowa:subst', 'w:prep'), 28442),
 (('mowa:subst', 'w:prep', 'usta:subst'), 13474),
 (('mowa:subst', 'w:prep', 'art:ign'), 12311),
 (('ustawa:subst', 'z:prep', 'dzień:subst'), 8588),
 (('właściwy:adj', 'do:prep', 'sprawa:subst'), 7966),
 (('minister:subst', 'właściwy:adj', 'do:prep'), 7888),
 (('w:prep', 'droga:subst', 'rozporządzenie:subst'), 4751),
 (('zastępować:fin', 'się:qub', 'wyraz:subst'), 3653),
 (('w:prep', 'ustawa:subst', 'z:prep'), 3646)]

In [53]:
number_of_tokens_clarin_trigrams = number_of_tokens_clarin
number_of_trigrams_clarin = sum(trigrams_clarin_clean_counter.values())

In [54]:
trigrams_filtered_clarin = defaultdict(int)

for k, v in trigrams_clarin_clean_counter.items():
    if v >= 5:
        trigrams_filtered_clarin[k] = v

In [55]:
pmi_trigrams_clarin = defaultdict(float)

for trigram, value in trigrams_filtered_clarin.items():
    denominator = 1
    for token in trigram:
        denominator *=  tokens_clarin_global_counter[token] / number_of_words_trigrams
    denominator *= 2 ** pmi_filtered[(trigram[0], trigram[1])]
    pmi_trigrams_clarin[trigram] = pmi_filtered[(trigram[0], trigram[1])] + np.log2((value/number_of_trigrams_clarin) / denominator)

pmi_trigrams_clarin_sorted = sorted(pmi_trigrams_clarin.items(), key=lambda x:x[1], reverse = True)

### Use PMI (with 5 occurrence threshold) to compute top 10 results for the trigrams. Devise a method for computing the values, based on the results for bigrams.

In [56]:
pmi_trigrams_sorted[:10]

[(('finałowego', 'turnieju', 'mistrzostw'), 37.040670670650584),
 (('profilem', 'zaufanym', 'epuap'), 36.7996625711468),
 (('cienką', 'sierścią', 'zwierzęcą'), 36.74111038879168),
 (('przedwczesnego', 'wyrębu', 'drzewostanu'), 36.61557950670782),
 (('centralnemu', 'biuru', 'antykorupcyjnemu'), 36.34879296601292),
 (('turnieju', 'mistrzostw', 'europy'), 36.27513592428761),
 (('potwierdzonym', 'profilem', 'zaufanym'), 36.252174775844296),
 (('szybkiemu', 'postępowi', 'technicznemu'), 36.12372641037815),
 (('piłce', 'nożnej', 'uefa'), 36.10461758743044),
 (('wypalonym', 'paliwem', 'jądrowym'), 35.901333989041696)]

In [57]:
pmi_trigrams_clarin_sorted[:10]

[(('porcelanowy:adj', 'młyn:subst', 'kulowy:adj'), 38.06904299083846),
 (('wymiennik:subst', 'przeponowy:adj', 'rurowy:adj'), 37.19832600778343),
 (('reakcja:subst', 'łańcuchowa:subst', 'rozszczepienie:subst'),
  36.36543599361868),
 (('piłka:subst', 'nożny:adj', 'uefa:subst'), 35.43943657506246),
 (('stany:subst', 'zjednoczyć:ppas', 'ameryka:subst'), 35.19832600778343),
 (('finałowy:adj', 'turniej:subst', 'mistrzostwa:subst'), 35.15666685614621),
 (('przedwczesny:adj', 'wyrąb:subst', 'drzewostan:subst'), 34.98400688698266),
 (('kurtka:subst', 'anorak:subst', 'etc:ign'), 34.84077400316534),
 (('mecz:subst', 'piłka:subst', 'nożny:adj'), 34.70247098089625),
 (('profil:subst', 'zaufany:adj', 'epuap:ign'), 34.61524324028049)]

### Create a table comparing the results for copora without and with tagging and lemmatization (separate table for bigrams and trigrams).

In [63]:
import pandas as pd 

In [73]:
df_pmi_filtered_sorted = pd.DataFrame.from_dict(pmi_filtered_sorted)
df_pmi_filtered_clarin_sorted = pd.DataFrame.from_dict(pmi_filtered_clarin_sorted)

In [76]:
df_bigrams = pd.concat([df_pmi_filtered_sorted, df_pmi_filtered_clarin_sorted], axis = 1)
df_bigrams.columns = ['Bigram_clean', 'PMI_clean', 'Bigram_clarin', 'PMI_clarin']

In [81]:
df_bigrams[:10]

,Bigram_clean,PMI_clean,Bigram_clarin,PMI_clarinn
0,"(świeckie, przygotowujące)",19.817294,"(teryto:ign, rialnego:ign)",19.823459
1,"(klęskami, żywiołowymi)",19.817294,"(młynek:subst, młotkowy:adj)",19.823459
2,"(ręcznego, miotacza)",19.817294,"(grzegorz:subst, schetyna:ign)",19.823459
3,"(stajnią, wyścigową)",19.817294,"(pasta:subst, emulsyjny:adj)",19.560424
4,"(otworami, wiertniczymi)",19.817294,"(odpowiedzieć:fin, dzialności:ign)",19.560424
5,"(teryto, rialnego)",19.817294,"(łańcuchowa:subst, rozszczepienie:subst)",19.560424
6,"(obcowania, płciowego)",19.817294,"(chrom:subst, sześciowartościowy:adj)",19.560424
7,"(nietykalność, cielesną)",19.817294,"(adam:subst, mickiewicz:subst)",19.560424
8,"(młyny, kulowe)",19.817294,"(piotrek:subst, trybunalski:adj)",19.338032
9,"(młynki, młotkowe)",19.817294,"(młyn:subst, kulowy:adj)",19.338032


In [78]:
df_pmi_trigrams_sorted = pd.DataFrame.from_dict(pmi_trigrams_sorted)
df_pmi_trigrams_clarin_sorted = pd.DataFrame.from_dict(pmi_trigrams_clarin_sorted)
df_trigrams = pd.concat([df_pmi_trigrams_sorted, df_pmi_trigrams_clarin_sorted], axis = 1)
df_trigrams.columns = ['Trigram_clean', 'PMI_clean', 'Trigram_clarin', 'PMI_clarin']

In [80]:
df_trigrams[:10]

,Trigram_clean,PMI_clean,Trigram_clarin,PMI_clarin
0,"(finałowego, turnieju, mistrzostw)",37.040671,"(porcelanowy:adj, młyn:subst, kulowy:adj)",38.069043
1,"(profilem, zaufanym, epuap)",36.799663,"(wymiennik:subst, przeponowy:adj, rurowy:adj)",37.198326
2,"(cienką, sierścią, zwierzęcą)",36.741110,"(reakcja:subst, łańcuchowa:subst, rozszczepien...",36.365436
3,"(przedwczesnego, wyrębu, drzewostanu)",36.615580,"(piłka:subst, nożny:adj, uefa:subst)",35.439437
4,"(centralnemu, biuru, antykorupcyjnemu)",36.348793,"(stany:subst, zjednoczyć:ppas, ameryka:subst)",35.198326
5,"(turnieju, mistrzostw, europy)",36.275136,"(finałowy:adj, turniej:subst, mistrzostwa:subst)",35.156667
6,"(potwierdzonym, profilem, zaufanym)",36.252175,"(przedwczesny:adj, wyrąb:subst, drzewostan:subst)",34.984007
7,"(szybkiemu, postępowi, technicznemu)",36.123726,"(kurtka:subst, anorak:subst, etc:ign)",34.840774
8,"(piłce, nożnej, uefa)",36.104618,"(mecz:subst, piłka:subst, nożny:adj)",34.702471
9,"(wypalonym, paliwem, jądrowym)",35.901334,"(profil:subst, zaufany:adj, epuap:ign)",34.615243


### Answer the following questions:
- Why do we have to filter the bigrams, rather than the token sequence?
- Which method works better for the bigrams and which for the trigrams?
- What types of expressions are discovered by the methods.
- Can you devise a different type of filtering that would yield better results?

Sekwencja tokenów nie niesie ze sobą wystarczającej informacji. Poprzez analizę ngramów możemy wyłuskać z tekstu inne znaczenia niż tylko dosłowne znaczenie pojedynczych słów, które często w połączeniu mają zupełnie inne znaczenie niż oddzielnie. Frazy mogą bowiem mieć charakter idiomatyczny lub też konkretyzować dane słowo.

Poprawę wyników mogłaby przynieść zmiana metryki w analizowanej metodzie. Zaproponować można tutaj metrykę z rodziny PMI^k
, która pozwala zredukować wpływ wyrażeń, które występują rzadziej.